### Import libraries

In [20]:
import pandas as pd
import numpy as np
import math
import re
import matplotlib
%matplotlib inline

### Loading data

In [21]:
DATA_FOLDER = 'data'
FOOD_FILE = 'food_data.csv'
COUNTRIES_FILE = 'countries_info.csv'
OVERWEIGHT_FILE = 'overweight.csv'
DIABETES_FILE = 'diabetes.csv'

In [22]:
all_datas = pd.read_csv(DATA_FOLDER + "/" + FOOD_FILE, sep='\t', low_memory=False)
countries_infos = pd.read_csv(DATA_FOLDER + "/" + COUNTRIES_FILE, sep=',', low_memory=False)
overweight = pd.read_csv(DATA_FOLDER + "/" + OVERWEIGHT_FILE, sep=',', low_memory=False)
diabetes = pd.read_csv(DATA_FOLDER + '/' + DIABETES_FILE, low_memory=False)

### Analysis of the different fields available in the dataset

We have to select only the features which is possible to help us to answer our questions

In [23]:
#for col in all_datas.columns.values:
#    print("- ", col)

We can deduce the usefull colums for our project:
 - product_name
 - sugars_100g
 - fat_100g
 - nutrition_grade_fr
 - countries
 - countries_tags


### Datasets clean
We clean our datasets. We select only the potential interesting fields. 

We do some operations on the countries tags to be able to parse the data. We also remove potential outliers.
In our case, some outliers may be having a larger sugar or fat content than 100g, since it defies the laws of physics :-p 

In [24]:
countries_infos = countries_infos[['COUNTRY_ALPHA2_CODE', 'COUNTRY_NAME']]

In [25]:
#usefull_datas = all_datas
to_keep = ['product_name', 'countries','countries_tags','sugars_100g','fat_100g','serving_quantity','energy_100g',
           'carbohydrates_100g','proteins_100g','salt_100g','sodium_100g','saturated-fat_100g',
           'ingredients_text', 'additives',  'fiber_100g','serving_size', 'additives_en']
usefull_datas = all_datas[to_keep]
#only do this for full na set
#usefull_datas.dropna(inplace=True)
usefull_datas['countries_tags'] = usefull_datas['countries_tags'].replace(regex=True, to_replace=r'\w\w:', value=r'')
usefull_datas['countries'].replace(regex=True, inplace=True, to_replace=r'\w\w:', value=r'')
usefull_datas['countries'].replace(regex=True, inplace=True, to_replace=r'-', value=r' ')
usefull_datas['countries'] = usefull_datas['countries'].apply(lambda x: str(x).lower())
usefull_datas = usefull_datas[usefull_datas['sugars_100g'] <= 100]
usefull_datas = usefull_datas[usefull_datas['fat_100g'] <= 100]
#get a feel of the data
usefull_datas.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is

,product_name,countries,countries_tags,sugars_100g,fat_100g,serving_quantity,energy_100g,carbohydrates_100g,proteins_100g,salt_100g,sodium_100g,saturated-fat_100g,ingredients_text,additives,fiber_100g,serving_size,additives_en
0,Vitória crackers,france,france,15.0,7.0,0,1569.0,70.1,7.8,1.400,0.551181,3.08,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,france,france,0.4,0.0,0,88.0,4.8,0.2,2.040,0.803150,0.00,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,france,france,3.9,0.3,0,134.0,5.3,0.9,0.420,0.165354,0.10,NaN,NaN,NaN,NaN,NaN
5,Fromage blanc aux myrtilles,france,france,16.3,4.9,0,540.0,16.3,4.4,0.250,0.098425,3.10,NaN,NaN,NaN,NaN,NaN
8,Baguette parisien,france,france,1.8,3.3,0,929.0,38.4,11.7,0.678,0.266929,2.10,NaN,NaN,NaN,NaN,NaN


We contruct a datastructure with the countries codes map to their names (in different languages), in order to group all the different ways the countries were written together

In [26]:

countries = []

for c in usefull_datas['countries_tags'].dropna():
    #split list of countries on the comma
    for country in c.split(','):
        country = country.lower()
        found = re.search('\w\w:(.+)', country, re.IGNORECASE)  
        if found:
            country = found.group(1)
        if country not in countries:
            countries.append(country)

countries_with_code = []
#apply lower case to country names
countries_infos['COUNTRY_NAME'] = countries_infos['COUNTRY_NAME'].apply(lambda x: str(x).lower())

for c in countries:
    is_present = False
    #loop over rows having matching country names with countries data
    for index, c_i in countries_infos[countries_infos['COUNTRY_NAME'] == c.lower()].iterrows():
        #add tuple containing country name and country code
        countries_with_code.append((c, c_i['COUNTRY_ALPHA2_CODE']))
        is_present = True
        break
    if not is_present:
        countries_with_code.append((c, "???"))

We select only the countries for which we have information. (They are contained in the open food database)

In [68]:
count = 0
proper_countries = []
for c in countries_with_code:
    if c[1] != "???":
        proper_countries.append(c)

In [69]:
to_append = [('united-states', 'US'), ('united-kingdom', 'UK'), ('french-polynesia', 'PF'),
             ('hong-kong', 'HK'), ('new-zealand', 'NZ'), ('new-caledonia', 'NC'),
             ('scotland','GB'), ('united-arab-emirates', 'AE'), ('etats-unis', 'US'),
             ('czech-republic', 'CZ'), ('south-africa', 'ZA'), ('quebec', 'CA'), 
             ('saint-pierre-and-miquelon', 'FR'),('french-guiana', 'FR'), ('dom-tom', 'FR'), ('south-korea', 'KR')]
for i in to_append:
    proper_countries.append(i)

In [70]:
proper_countries

[('france', 'FR'),
 ('canada', 'CA'),
 ('oman', 'OM'),
 ('germany', 'DE'),
 ('belgium', 'BE'),
 ('netherlands', 'NL'),
 ('mexico', 'MX'),
 ('australia', 'AU'),
 ('switzerland', 'CH'),
 ('egypt', 'EG'),
 ('italy', 'IT'),
 ('spain', 'ES'),
 ('saudi-arabia', 'SA'),
 ('iraq', 'IQ'),
 ('sverige', 'SE'),
 ('sweden', 'SE'),
 ('vietnam', 'VN'),
 ('singapore', 'SG'),
 ('thailand', 'TH'),
 ('greece', 'GR'),
 ('panama', 'PA'),
 ('malaysia', 'MY'),
 ('luxembourg', 'LU'),
 ('lebanon', 'LB'),
 ('denmark', 'DK'),
 ('guyana', 'GY'),
 ('serbia', 'RS'),
 ('martinique', 'MQ'),
 ('barbados', 'BB'),
 ('ireland', 'IE'),
 ('aruba', 'AW'),
 ('bahrain', 'BH'),
 ('taiwan', 'TW'),
 ('cuba', 'CU'),
 ('portugal', 'PT'),
 ('kuwait', 'KW'),
 ('austria', 'AT'),
 ('japan', 'JP'),
 ('israel', 'IL'),
 ('royaume-uni', 'GB'),
 ('brazil', 'BR'),
 ('morocco', 'MA'),
 ('guinea', 'GN'),
 ('china', 'CN'),
 ('algeria', 'DZ'),
 ('mauritius', 'MU'),
 ('indonesia', 'ID'),
 ('turkey', 'TR'),
 ('belgien', 'BE'),
 ('deutschland', 'DE

When a rows mention many countries we explode the dataframe to get one line for each country.

In [42]:
def explode(df):
    rows = []
    for index, data in df.iterrows():
        countries = str(data['countries_tags']).split(',')
        for country in countries:
            data['countries'] = country
            rows.append(data)
    return pd.DataFrame(rows)

In [43]:
exploded_data = explode(usefull_datas)

In [54]:
def get_tag(row):
    for c in proper_countries:
        if c[0] == row['countries']:
            row['countries_tags'] = c[1]
            return row
    row['countries_tags'] = "TBD"
    return row


In [71]:
parsed_datas = exploded_data.apply(get_tag, axis=1)

In [78]:
parsed_datas[parsed_datas['countries_tags'] == 'TBD']['countries']

16164                     nan
18679                     nan
28022                     nan
29679                     nan
39487                      en
39487                      en
39656                      en
39656                      en
39656                      en
60736                      en
60736                      en
60738                      en
60738                      en
62077                     nan
66242                     nan
67677          estadps-unidos
67677          estadps-unidos
71979                     nan
72989                     nan
74551                     nan
76487                     nan
76636                 pour-la
76636                 pour-la
76636                 pour-la
109135                     en
109135                     en
112565                    nan
116080         european-union
117296                     en
117296                     en
                 ...         
677796                     en
679017                     en
679017    

In [77]:
parsed_datas[parsed_datas['countries_tags'] == 'TBD']['countries'].value_counts()

en                                     645
nan                                    127
reino-unido                             13
vereinigtes-konigreich                  13
european-union                          10
suede                                   10
republic-of-macedonia                    9
grece                                    8
europe                                   8
verenigd-koninkrijk                      7
republic-of-the-congo                    7
franca                                   6
polyensie-francaise                      5
tahiti                                   5
carrefour                                5
magyarorszag                             5
cote-d-ivoire                            5
rumanien                                 4
cee                                      4
world                                    4
draveil                                  4
صنعاء                                    4
england                                  4
costa-rica 

In [73]:
parsed_datas['countries_tags'].value_counts()

FR     360855
US     162820
CH      19013
DE      13916
ES       6635
UK       6297
IT       2035
MX       1524
BE       1421
SE       1271
NL       1194
TBD      1009
PT        911
CA        775
AU        678
RS        595
RO        587
RE        531
IE        433
LU        415
GP        402
HU        371
PF        361
MQ        276
MA        275
NC        230
PL        203
RU        199
NZ        187
TN        173
        ...  
AD          5
AL          4
VU          4
ME          4
SZ          4
VE          4
PK          3
MO          3
AZ          3
EE          3
GN          3
KE          3
IQ          3
ML          2
NG          2
CM          2
PA          2
LY          2
MD          2
EC          2
MZ          2
HR          2
NE          2
FJ          2
TG          2
GI          1
GT          1
AW          1
HT          1
DM          1
Name: countries_tags, Length: 120, dtype: int64

### Get results
Now we compute result from the parsed datasets. We would like to get the mean average of sugar and fat per 100g per country.

In [ ]:
grouped = parsed_datas.groupby(['countries_tags'])
means = grouped.mean()
means

In [ ]:
grouped = parsed_datas.groupby(['countries_tags'])
counted = grouped.count()

In [ ]:
sugar = means[['sugars_100g']].dropna()
#remove outliers
sugar = sugar[sugar['sugars_100g'] <= 100]
#we would like to have countries with a big enough sample of values in the database, we choose 20 for now
sugar = sugar[counted['sugars_100g'] > 20]
#sort values
sugar.sort_values(by=['sugars_100g'], ascending=False, inplace=True)

In [ ]:
sugar.plot.bar(figsize=(15,10))

We can see which countries consume the most sugars according to our datas.

In [ ]:
#remove null values
fat = means[['fat_100g']].dropna()
#remove outliers
fat = fat[fat['fat_100g'] <= 100]
#remove values with small sample
fat = fat[counted['fat_100g'] > 20]
#sort
fat.sort_values(by=['fat_100g'], ascending=False, inplace=True)
fat.plot.bar(figsize=(15,10))

In [ ]:
def corr_scatter(df1, df2, merge_on, x_, y_, title_):
    merge = pd.merge(df1, df2, on=merge_on)
    merge.plot(kind='scatter',x=x_,y=y_, title=title_)
    print('Pearson correlation: ', merge.corr(method='pearson').iloc[0][1])
    print('Kendall correlation: ', merge.corr(method='kendall').iloc[0][1])
    print('Spearman correlation: ', merge.corr(method='spearman').iloc[0][1])

In [ ]:
corr_scatter(sugar, fat, 'countries_tags', 'sugars_100g', 'fat_100g', 'Sugar vs. Fat')

We can see which countries consume the most fat according to our datas. 
We can guess some correlations, for instance (Slovakia are on the top of the ranking for each aspect) and some independant values (Germany use a lot of fat but not so much sugar), but we would need to calculate to be sure (next milestone)

We get the union of the sugar and fat datas to filter some other results.

In [ ]:
indexes = pd.DataFrame(np.concatenate((sugar.index.values,fat.index.values)))
indexes = indexes.drop_duplicates()

In [ ]:
overweight_datas = overweight.copy()
#choose data for a specific year. Unfortunately our dataset only goes up until 2016, so we choose that year.
overweight_datas = overweight_datas[overweight_datas['Year'] == 2016]
#get country codes
overweight_datas['Entity'] = overweight_datas['Entity'].apply(lambda x: get_tag(str(x).lower().strip()))
#remove rows with no country code
overweight_datas = overweight_datas[overweight_datas['Entity'] != 'TBD']
#set index so it's easier to plot
overweight_datas = overweight_datas.set_index(['Entity'])
#keep relevant column
overweight_datas = overweight_datas[['Indicator']]
#sort values
overweight_datas = overweight_datas.sort_values(by=['Indicator'], ascending=False)
overweight_datas = overweight_datas[overweight_datas.index.isin(indexes[0])]
overweight_datas = overweight_datas.rename(columns={'Indicator':'Overweight Prevalence'})
overweight_datas.index.name = 'countries_tags'
overweight_datas.head()

In [ ]:
overweight_datas.plot.bar(figsize=(20,10))

In [ ]:
corr_scatter(fat, overweight_datas,'countries_tags', 'Overweight Prevalence', 'fat_100g', 'Overweight vs. Fat')

In [ ]:
corr_scatter(sugar, overweight_datas,'countries_tags', 'Overweight Prevalence', 'sugars_100g', 'Overweight vs. Sugar')

As we can see : It is difficult to guess a correlation between the previous obsrvations (sugar & fat) and the obesity percentage displayed above. 
For example the slovakia is in the mean (for the obesity) but not really for the sugar & fat. That is only a guess we have to provide a scatter plot to proove it.

In [ ]:
#drop some useless columns
to_drop=['prev', 'Unnamed: 5']
diab = diabetes.drop(to_drop, axis=1)
#select relevant columns
diab=diab[['country/territory', 'Diabetes [18-99] national prevalence[%]']]
#convert string column to numeric, in order to plot it, remove % at the end
diab['Diabetes [18-99] national prevalence[%]'] = pd.to_numeric(diab['Diabetes [18-99] national prevalence[%]'].str[:-1])
#get country codes
diab['country/territory'] = diab['country/territory'].apply(lambda x: get_tag(str(x).lower().strip()))
#remove rows without country codes
diab = diab[diab['country/territory'] != 'TBD']
#set index for more agreable plotting
diab.set_index('country/territory', inplace=True)
diab = diab[diab.index.isin(indexes[0])]
#sort values
diab= diab.sort_values(by=['Diabetes [18-99] national prevalence[%]'],ascending=False)
diab = diab.rename(columns={'Diabetes [18-99] national prevalence[%]':'Diabetes Prevalence'})
diab.index.name = 'countries_tags'
diab.plot.bar(figsize=(20,10))

In [ ]:
corr_scatter(fat, diab,'countries_tags', 'Diabetes Prevalence', 'fat_100g', 'Diabetes vs. Fat')

In [ ]:
corr_scatter(sugar, diab,'countries_tags', 'Diabetes Prevalence', 'sugars_100g', 'Diabetes vs. Sugar')

In [ ]:
corr_scatter(overweight_datas, diab,'countries_tags', 'Overweight Prevalence', 'Diabetes Prevalence', 'Overweight vs. Diabetes')

Like for the obesity : it is difficult to guess a correlation between the previous obsrvations (sugar & fat) and the diabetes percentage display above. Same example : slovakia is in just after the middle but not really for the sugar & fat. Once more that is only a guess we have to provide a scatter plot to proove it.

### Summarize
We selected the datas who have an interest for us. We parse them to be able to get some results from it. We already get some results concerning alimentation quality of different countries. The really next step is to prove if a correlation exists between our results. (We have some guesses, but we have to provide a scatter plot and a correlation coefficient to proove it)

### What comes next

First, we reoriented our approach of our study. 
We decided to globalize our approach and not to focus our work only on the european countries. 
Indeed not too many countries have relevant data so we can include all of them.

We spent a lot of time on cleaning the datasets, so we focused our milestone on the countries part. 
We now will be able to have more relevant results, such as a visualization of the correlation (or not) between features.
We will focus on the food quality in the next steps, such as finding which particular foods have a higher sugar or fat content, instead of just comparing the countries. 

#### Research questions 
As we discussed above we did not yet work on the food quality, so we are not able to answer our research questions for the moment. We have to extract more data and try to find a correlation with the ones we already have. 

In [ ]:
data = all_datas.copy()
data['countries_tags'].replace(regex=True, inplace=True, to_replace=r'\w\w:', value=r'')
data = data[data['sugars_100g'] <= 100]
data = data[data['fat_100g'] <= 100]

In [ ]:
not_na = []
for c in data.columns.values: 
    count = data[c].notna().sum()
    not_na.append((c, count))
df = pd.DataFrame(not_na, columns=['Name','Not_NA_Count'])
df.sort_values(by='Not_NA_Count', inplace=True, ascending=False)
df = df.reset_index(drop=True)
df.plot(kind='bar',x='Name', y='Not_NA_Count', figsize=(18,10))

In [ ]:
df_2 = df[df['Not_NA_Count'] > 10000]
df_2.plot(kind='bar',x='Name', y='Not_NA_Count', figsize=(18,10))

In [ ]:
to_keep = ['sugars_100g','serving_quantity','fat_100g','energy_100g','carbohydrates_100g','proteins_100g',
           'product_name','salt_100g','sodium_100g','saturated-fat_100g','ingredients_that_may_be_from_palm_oil_n',
           'ingredients_text','ingredients_from_palm_oil_n', 'additives_n','additives','fiber_100g','serving_size',
           'additives_en','additives_tags','calcium_100g','vitamin-c_100g','cholesterol_100g',
           'iron_100g','trans-fat_100g','vitamin-a_100g','allergens','traces_en',
           'manufacturing_places','potassium_100g','vitamin-b1_100g','vitamin-b2_100g']

to_keep_red = ['sugars_100g','serving_quantity','fat_100g','energy_100g','carbohydrates_100g','proteins_100g',
           'product_name','salt_100g','sodium_100g','saturated-fat_100g','ingredients_that_may_be_from_palm_oil_n',
           'ingredients_text','ingredients_from_palm_oil_n', 'additives_n','additives','fiber_100g','serving_size',
           'additives_en']

to_keep_red_2 = ['sugars_100g','serving_quantity','fat_100g','energy_100g','carbohydrates_100g','proteins_100g',
                 'product_name','salt_100g','sodium_100g','saturated-fat_100g','ingredients_text', 'additives',
                 'fiber_100g','serving_size', 'additives_en']

In [ ]:
data = data[to_keep_red_2]
data

In [ ]:
parsed_datas.sort_values(by='sugars_100g', ascending=False)[0:100]